In [1]:
# conda install -y gensim
# run cell
# conda uninstall -y boto
# conda install -y boto

import glob
import os
import numpy as np
import pandas as pd
import pymorphy2
import re
from tqdm import tqdm
import _pickle as cPickle

## Настройки

In [182]:
# Путь к папке с сохраненными данными
data_folder = 'rutent2'

# Путь к папке с предобученной моделью
# rutent2_2: Presicion: 0.9664723032069971, Recall: 0.9477445997458704, F-score: 0.9570168404170009
# rutent2_3: Presicion: 0.9608867775138559, Recall: 0.9637865311308768, F-score: 0.9623344699072238
# ententen13_1: Presicion: 0.5797101449275363, Recall: 0.5882352941176471, F-score: 0.583941605839416
# ententen13_2: Presicion: 0.6311475409836066, Recall: 0.5661764705882353, F-score: 0.5968992248062016
# (на чисто тестовой: Presicion: 0.6280487804878049, Recall: 0.5786516853932584, F-score: 0.6023391812865497)
model_folder = 'rutent2_3'

MAX_SEQ_LENGTH = 30  # максимальное количество слов в предложении

# Настройки сети
LEARNING_RATE = .0001
TRAIN_SPLIT = 0.7  # доля обучающей выборки

## Загрузка тестовой выборки из файлов

In [3]:
X_test = cPickle.load(open(os.path.join(data_folder, "X_test.pkl"), "rb"))
targets = cPickle.load(open(os.path.join(data_folder, "y_test.pkl"), "rb"))
texts = cPickle.load(open(os.path.join(data_folder, "texts.pkl"), "rb"))

In [4]:
print("X_test shape:", X_test.shape)
print("y_test shape:", targets.shape)
print("texts length:", len(texts))

X_test shape: (65051, 30, 200)
y_test shape: (65051, 30, 2)
texts length: 216835


## Predict

In [5]:
from keras.optimizers import Adagrad
from keras.models import model_from_json
from sklearn.metrics import precision_score, recall_score, f1_score

/home/o_sopilniak/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/o_sopilniak/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [6]:
adagrad = Adagrad(lr=.01, epsilon=None, decay=1e-6)

In [244]:
# Загрузим модель

json_file = open(os.path.join(model_folder, 'model.json'), 'r')

loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)

model.load_weights(os.path.join(model_folder, 'model.h5'))
model.compile(loss='categorical_crossentropy', optimizer=adagrad, metrics=['categorical_accuracy'])  # 'adam', 'accuracy'

In [245]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        (None, 30, 200)           0         
_________________________________________________________________
bidirectional_15 (Bidirectio (None, 30, 128)           135680    
_________________________________________________________________
dense_14 (Dense)             (None, 30, 2)             258       
_________________________________________________________________
activation_14 (Activation)   (None, 30, 2)             0         
Total params: 135,938
Trainable params: 135,938
Non-trainable params: 0
_________________________________________________________________


In [204]:
# %%time

# scores = model.evaluate(X_test, targets, verbose=0)

In [10]:
scores

[0.0013932744138250752, 0.9676125902552031]

In [301]:
test_start = 151784  # X.shape[0]

# Меньше какой вероятности отрицательного класса говорить, что принадлежит положительному классу
# POSITIVE_ANSWER_THRESHOLD = 0.665
POSITIVE_ANSWER_THRESHOLD = 0.5
# POSITIVE_ANSWER_THRESHOLD = 4e-1
# POSITIVE_ANSWER_THRESHOLD = 9.1e-1

## Проверка

In [291]:
for i, text in zip(range(len(texts[test_start:test_start + 1000])), texts[test_start:test_start + 1000]):
    if "не | будет" in text:
        print(i)

281
389


In [292]:
ind = 281
predicted_values_ = model.predict(X_test[ind:ind + 1])

words = texts[test_start + ind].split(" | ")[:-1][:MAX_SEQ_LENGTH]

predicted_values_[0][:len(words)]

array([[8.3502790e-08, 9.9999988e-01],
       [5.6741430e-07, 9.9999940e-01],
       [8.1057552e-09, 1.0000000e+00],
       [5.1337907e-09, 1.0000000e+00],
       [1.4514098e-08, 1.0000000e+00],
       [3.1596915e-07, 9.9999964e-01],
       [4.0412942e-06, 9.9999595e-01],
       [3.4738708e-08, 1.0000000e+00],
       [1.4269006e-08, 1.0000000e+00],
       [1.2386043e-08, 1.0000000e+00],
       [1.5163545e-08, 1.0000000e+00],
       [1.1104365e-06, 9.9999893e-01],
       [8.6119837e-01, 1.3880165e-01],
       [1.2864594e-05, 9.9998713e-01],
       [3.5456666e-01, 6.4543331e-01]], dtype=float32)

In [293]:
predicted_final = []

for value in predicted_values_[0]:
    if value[0] > POSITIVE_ANSWER_THRESHOLD:
        predicted_final.append([1., 0.])
    else:
        predicted_final.append([0., 1.])
        
np.array(predicted_final)[:, 0][:len(words)]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.])

In [294]:
real_final = targets[ind].tolist()
np.array(real_final)[:, 0][:len(words)]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0])

In [295]:
real_length = len(real_final)
f1_score(np.array(predicted_final)[:, 0][:len(words)], np.array(real_final)[:, 0][:len(words)], average='binary')

1.0

In [296]:
texts[test_start + ind]

'Как | говорится | в | пресс-релизе | АО | в целом | повестка | дня | собрания | которое | состоится | апреля | изменена | не | будет | '

## Валидация

In [253]:
predicted_values = model.predict(X_test)

In [302]:
error_sentences_indexes = []

cases_dict = {
    'after_no_indexes_tp': [],
    'after_no_indexes_fn': [],
    'after_no_indexes_fp': [],
    'after_no_indexes_tn': [],
    'before_no_indexes_tp': [],
    'before_no_indexes_fn': [],
    'before_no_indexes_fp': [],
    'before_no_indexes_tn': [],
    'after_1_no_tp': [],
    'after_1_no_fn': [],
    'after_1_no_fp': [],
    'after_1_no_tn': [],
    'after_2_3_no_tp': [],
    'after_2_3_no_fn': [],
    'after_2_3_no_fp': [],
    'after_2_3_no_tn': [],
    'are_not_tp': [],
    'are_not_fn': [],
    'are_not_fp': [],
    'are_not_tn': []
}

overall_predicted = []
overall_real = []

# Для каждого предложения считаем метрики
for (i, predicted) in tqdm(zip(range(len(predicted_values)), predicted_values)):

    predicted_final = []

    for value in predicted:
        if value[0] > POSITIVE_ANSWER_THRESHOLD:
            predicted_final.append([1., 0.])
        else:
            predicted_final.append([0., 1.])

    real_final = targets[i].tolist()

    real_length = len(real_final)

    if real_length > 0:

        words = texts[test_start + i].split(" | ")[:-1][:MAX_SEQ_LENGTH]

        predicted = np.array(predicted_final)[:, 0][:len(words)]
        real = np.array(real_final)[:, 0][:len(words)]

        overall_predicted.extend(predicted)
        overall_real.extend(real)

        # Сохраним номера предложений ответами в разных отдельных случаях
        if "не" in words:

            index_of_no = words.index("не")

            try:
                # Сразу после "не"
                if real[index_of_no + 1] == 1:
                    if predicted[index_of_no + 1] == 1:
                        cases_dict['after_no_indexes_tp'].append(test_start + i)
                    else:
                        cases_dict['after_no_indexes_fn'].append(test_start + i)
                else:
                    if predicted[index_of_no + 1] == 1:
                        cases_dict['after_no_indexes_fp'].append(test_start + i)
                    else:
                        cases_dict['after_no_indexes_tn'].append(test_start + i)

                # Сразу перед "не"
                if real[index_of_no - 1] == 1:
                    if predicted[index_of_no - 1] == 1:
                        cases_dict['before_no_indexes_tp'].append(test_start + i)
                    else:
                        cases_dict['before_no_indexes_fn'].append(test_start + i)
                else:
                    if predicted[index_of_no - 1] == 1:
                        cases_dict['before_no_indexes_fp'].append(test_start + i)
                    else:
                        cases_dict['before_no_indexes_tn'].append(test_start + i)

                # Через 1 после "не"
                if real[index_of_no + 2] == 1:
                    if predicted[index_of_no + 2] == 1:
                        cases_dict['after_1_no_tp'].append(test_start + i)
                    else:
                        cases_dict['after_1_no_fn'].append(test_start + i)
                else:
                    if predicted[index_of_no + 2] == 1:
                        cases_dict['after_1_no_fp'].append(test_start + i)
                    else:
                        cases_dict['after_1_no_tn'].append(test_start + i)

                # Через 2 или 3 после "не"
                if real[index_of_no + 3] == 1:
                    if predicted[index_of_no + 3] == 1:
                        cases_dict['after_2_3_no_tp'].append(test_start + i)
                    else:
                        cases_dict['after_2_3_no_fn'].append(test_start + i)

                else:
                    if predicted[index_of_no + 3] == 1:
                        cases_dict['after_2_3_no_fp'].append(test_start + i)
                    else:
                        cases_dict['after_2_3_no_tn'].append(test_start + i)                  

                if real[index_of_no + 4] == 1:
                    if predicted[index_of_no + 4] == 1:
                        cases_dict['after_2_3_no_tp'].append(test_start + i)
                    else:
                        cases_dict['after_2_3_no_fn'].append(test_start + i)
                else:
                    if predicted[index_of_no + 4] == 1:
                        cases_dict['after_2_3_no_fp'].append(test_start + i)
                    else:
                        cases_dict['after_2_3_no_tn'].append(test_start + i)

            except IndexError:
                pass

        # "Нет"
        if " нет " in texts[test_start + i]:
            words = texts[test_start + i].split(" | ")[:MAX_SEQ_LENGTH]
            index_of_not = words.index("нет")

            try:
                if real[index_of_not] == 1:
                    if predicted[index_of_not] == 1:
                        cases_dict['are_not_tp'].append(test_start + i)
                    else:
                        cases_dict['are_not_fn'].append(test_start + i)

            except IndexError:
                pass

        # Сохраним номера предложений, для которых предсказанное и реальное значение не совпали
        if not np.array_equal(predicted, real):
            error_sentences_indexes.append(test_start + i)

65051it [00:09, 6746.15it/s]


In [303]:
print("Presicion: {}, Recall: {}, F-score: {}".format(precision_score(overall_real, overall_predicted, average='binary'), 
                                                      recall_score(overall_real, overall_predicted, average='binary'), 
                                                      f1_score(overall_real, overall_predicted, average='binary')))

Presicion: 0.9578997919001121, Recall: 0.9504447268106735, F-score: 0.9541576975205295


In [283]:
for key in cases_dict.keys():
    print("{}: {}".format(key, len(cases_dict[key])))

after_no_indexes_tp: 4946
after_no_indexes_fn: 63
after_no_indexes_fp: 162
after_no_indexes_tn: 287
before_no_indexes_tp: 22
before_no_indexes_fn: 31
before_no_indexes_fp: 8
before_no_indexes_tn: 5397
after_1_no_tp: 199
after_1_no_fn: 58
after_1_no_fp: 57
after_1_no_tn: 4653
after_2_3_no_tp: 57
after_2_3_no_fn: 28
after_2_3_no_fp: 11
after_2_3_no_tn: 9025
are_not_tp: 218
are_not_fn: 10
are_not_fp: 0
are_not_tn: 0


In [284]:
i = 0
values = []

for key in cases_dict.keys():
    if i == 3:
        precision = values[0] / (values[0] + values[2])
        recall = values[0] / (values[0] + values[1])
        f_measure = 2 * precision * recall / (precision + recall)
        values = []
        i = 0
        print("{}: precision {}, recall {}, f-score {}".format(key, precision, recall, f_measure))
    else:
        i += 1
        values.append(len(cases_dict[key]))

after_no_indexes_tn: precision 0.9682850430696947, recall 0.9874226392493511, f-score 0.9777602055945438
before_no_indexes_tn: precision 0.7333333333333333, recall 0.41509433962264153, f-score 0.5301204819277109
after_1_no_tn: precision 0.77734375, recall 0.77431906614786, f-score 0.7758284600389864
after_2_3_no_tn: precision 0.8382352941176471, recall 0.6705882352941176, f-score 0.7450980392156863
are_not_tn: precision 1.0, recall 0.956140350877193, f-score 0.9775784753363228


## Baseline

In [43]:
morph = pymorphy2.MorphAnalyzer()

In [44]:
# Настройки бейзлайна

LEMMAS_TO_EXCLUDE = ['быть']  # Леммы слов, которые надо выкинуть
POS_TO_EXCLUDE = ['PREP']  # Части речи, которые надо выкинуть

In [45]:
predicted_values.shape

(65051, 30, 2)

In [46]:
'''
Baseline-решение
Считаем, что если слово находится сразу после "не" и "нет", то оно лежит в нашем negation scope
'''

def baseline(start_index=0):
    
    is_prev_cue = False
    tp, fp, tn, fn = 0, 0, 0, 0
    
    overall_labeled = 0
    
    # Контексты неудавшихся примеров
    fp_context = []
    fn_context = []
    fp_difference = []
    fn_difference = []
    
    cases_dict_baseline = {
        'after_no_indexes_tp': [],
        'after_no_indexes_fn': [],
        'after_no_indexes_fp': [],
        'after_no_indexes_tn': [],
        'after_1_no_tp': [],
        'after_1_no_fn': [],
        'after_1_no_fp': [],
        'after_1_no_tn': [],
        'after_2_3_no_tp': [],
        'after_2_3_no_fn': [],
        'after_2_3_no_fp': [],
        'after_2_3_no_tn': [],
        'are_not_tp': [],
        'are_not_fn': [],
        'are_not_fp': [],
        'are_not_tn': []
    }
    
    for (i, sentence) in zip(range(len(texts[start_index:])), texts[start_index:]):
        
        sentence = sentence.split(" | ")[:-1][:MAX_SEQ_LENGTH]
        excluded_count = 0
        
        for (word_ind, word) in zip(range(len(sentence)), sentence):
            
            if (word.lower() == 'не'):
                is_prev_cue = True
                continue

            # Разбираем случаи
            if is_prev_cue or word.lower() == 'нет':

                # Проверяем, не нужно ли выкинуть слово
                if (morph.parse(word)[0].normal_form in LEMMAS_TO_EXCLUDE):
                    excluded_count += 1
                    continue

                # Проверяем часть речи
                if (morph.parse(word)[0].tag.POS in POS_TO_EXCLUDE):
                    excluded_count += 1
                    continue

                if (targets[i][word_ind][0] == 0):
                    fp += 1
                    fp_context.append(sentence)
                    
                    # Если сетка в этом случае ответила правильно, записываем
                    if start_index + i not in error_sentences_indexes:
                        
                        net_answer = predicted_values[i]
                        output = ""
                        
                        for (out_word_ind, output_word) in zip(range(len(sentence)), sentence):
        
                            if output_word == "":
                                continue

                            if net_answer[out_word_ind][1] < POSITIVE_ANSWER_THRESHOLD and targets[i][out_word_ind][0] == 1:
                                output += "***" + output_word + "*** "
                                continue

                            if net_answer[out_word_ind][1] < POSITIVE_ANSWER_THRESHOLD:
                                output += "**" + output_word + "** "
                                continue

                            if targets[i][out_word_ind][0] == 1:
                                output += "*" + output_word + "* "
                                continue

                            output += output_word + " "
                        
                        fp_difference.append(output)
                    
                    # Сохраняем номера предложений в различных случаях
                    if excluded_count == 0:
                        cases_dict_baseline['after_no_indexes_fp'].append(test_start + i)
                    
                    if excluded_count == 1:
                        cases_dict_baseline['after_1_no_fp'].append(test_start + i)
                    
                    if excluded_count == 2 or excluded_count == 3:
                        cases_dict_baseline['after_2_3_no_fp'].append(test_start + i)
                        
                    if word.lower() == 'нет':
                        cases_dict_baseline['are_not_fp'].append(test_start + i)
                    
                else:
                    tp += 1
                    overall_labeled += 1
                    
                    # Сохраняем номера предложений в различных случаях
                    if excluded_count == 0:
                        cases_dict_baseline['after_no_indexes_tp'].append(test_start + i)
                    
                    if excluded_count == 1:
                        cases_dict_baseline['after_1_no_tp'].append(test_start + i)
                    
                    if excluded_count == 2 or excluded_count == 3:
                        cases_dict_baseline['after_2_3_no_tp'].append(test_start + i)
                        
                    if word.lower() == 'нет':
                        cases_dict_baseline['are_not_tp'].append(test_start + i)
                    
                    
            else:
                if (targets[i][word_ind][0] == 0):
                    tn += 1
                    
                    # Сохраняем номера предложений в различных случаях
                    if excluded_count == 0:
                        cases_dict_baseline['after_no_indexes_tn'].append(test_start + i)
                    
                    if excluded_count == 1:
                        cases_dict_baseline['after_1_no_tn'].append(test_start + i)
                    
                    if excluded_count == 2 or excluded_count == 3:
                        cases_dict_baseline['after_2_3_no_tn'].append(test_start + i)
                        
                    if word.lower() == 'нет':
                        cases_dict_baseline['are_not_tn'].append(test_start + i)
                        
                else:
                    fn += 1
                    overall_labeled += 1
                    
                    # Сохраняем номера предложений в различных случаях
                    if excluded_count == 0:
                        cases_dict_baseline['after_no_indexes_fn'].append(test_start + i)
                    
                    if excluded_count == 1:
                        cases_dict_baseline['after_1_no_fn'].append(test_start + i)
                    
                    if excluded_count == 2 or excluded_count == 3:
                        cases_dict_baseline['after_2_3_no_fn'].append(test_start + i)
                        
                    if word.lower() == 'нет':
                        cases_dict_baseline['are_not_fn'].append(test_start + i)
                        
                    fn_context.append(sentence)
                    
                    # Если сетка в этом случае ответила правильно, записываем
                    if start_index + i not in error_sentences_indexes:
                        
                        net_answer = predicted_values[i]
                        output = ""
                        
                        for (out_word_ind, output_word) in zip(range(len(sentence)), sentence):
        
                            if output_word == "":
                                continue

                            if net_answer[out_word_ind][1] < POSITIVE_ANSWER_THRESHOLD and targets[i][out_word_ind][0] == 1:
                                output += "***" + output_word + "*** "
                                continue

                            if net_answer[out_word_ind][1] < POSITIVE_ANSWER_THRESHOLD:
                                output += "**" + output_word + "** "
                                continue

                            if targets[i][out_word_ind][0] == 1:
                                output += "*" + output_word + "* "
                                continue

                            output += output_word + " "
                        
                        fn_difference.append(output)

            is_prev_cue = False

        is_prev_cue = False
    
    # Считаем метрики
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f_measure = 2 * precision * recall / (precision + recall)
    
    return precision, recall, f_measure, fp_context, fn_context, fp_difference, fn_difference, \
        cases_dict_baseline, overall_labeled

In [47]:
precision, recall, f_measure, fp_context, fn_context, fp_difference, fn_difference, cases_dict_baseline, \
    overall_labeled = baseline(test_start)
print("precision={}, recall={}, F-score={}".format(precision, recall, f_measure))

precision=0.9299293008641005, recall=0.9692156541673489, F-score=0.9491661321359846


In [48]:
overall_labeled

6107

In [49]:
for key in cases_dict_baseline.keys():
    print("{}: {}".format(key, len(cases_dict_baseline[key])))

after_no_indexes_tp: 5604
after_no_indexes_fn: 139
after_no_indexes_fp: 175
after_no_indexes_tn: 805159
after_1_no_tp: 307
after_1_no_fn: 48
after_1_no_fp: 245
after_1_no_tn: 4144
after_2_3_no_tp: 8
after_2_3_no_fn: 1
after_2_3_no_fp: 26
after_2_3_no_tn: 229
are_not_tp: 245
are_not_fn: 0
are_not_fp: 34
are_not_tn: 0


In [50]:
i = 0
values = []

for key in cases_dict_baseline.keys():
    if i == 3:
        precision = values[0] / (values[0] + values[2])
        recall = values[0] / (values[0] + values[1])
        f_measure = 2 * precision * recall / (precision + recall)
        values = []
        i = 0
        print("{}: precision {}, recall {}, f-score {}".format(key, precision, recall, f_measure))
    else:
        i += 1
        values.append(len(cases_dict_baseline[key]))

after_no_indexes_tn: precision 0.9697179442810174, recall 0.9757966219745777, f-score 0.9727477868425621
after_1_no_tn: precision 0.5561594202898551, recall 0.8647887323943662, f-score 0.6769570011025358
after_2_3_no_tn: precision 0.23529411764705882, recall 0.8888888888888888, f-score 0.3720930232558139
are_not_tn: precision 0.8781362007168458, recall 1.0, f-score 0.9351145038167938


In [51]:
# не после "не", размечено, нейронка ответила правильно
fn_difference[:10]

['***значит*** что мы рекомендуем массовое банкротство достаточно одного прецедента чтобы директора поняли что надо платить по счетам сказал Уиллоби По его словам проблема не ***возврата*** кредитов стала одной из самых ',
 '***менее*** дней ',
 'Процентные ставки по валютным вкладам ***пересматриваться*** не будут сказал представитель Нац банка ',
 'МОСКВА июн Рейтер Аукцион по продаже процентов плюс одной акции АО Связьинвест на нынешней неделе ***объявлен*** не будет сказала Рейтер пресс-секретарь вице премьера председателя ГКИ РФ Виктория Вергельская ',
 'а потом подключились продавцы и началась продажа которая была не по всему ***рынку*** достаточно рваная Возможно это связано с прибалтийскими банками корсчета которых в петербургских банках арестованы налоговой полицией сказал ',
 'Российская сторона в свою очередь предлагала им участие в приватизации Таганрогского морского порта однако договоренности ***достигнуто*** не было из-за того что этот порт недавно пострадал в результате

In [52]:
# после "не", не размечено, нейронка ответила правильно
fp_difference

['длинные бумаги что их можно взять только для диверсификации портфеля ценных бумаг Я думаю что в ближайшее время не ***будет*** больших эмиссий этих бумаг как в свое время было с ',
 'У украинских банков не ***было*** столько долларов это выставлялись деньги нерезидентов и курс ушел а сегодня все возвращается обратно сказал Тимофеев ',
 'Вечером реальный рынок немного присел соответственно и фьючерс немного упал Может быть это снижение цен в РТС связано с тем что не ***было*** западных заказов сказал Александр Кареевский из компании Индекс ',
 'По его словам у ЗиЛа не ***было*** прибыли и в году ',
 'Это был рискованный шаг потому что как я думаю у Минфина не ***было*** сто процентной уверенности что он сможет взять эти деньги на торговой сессии сказал Орехов ',
 'Победы ОНЭКСИМа ожидал весь рынок это ни для кого не ***было*** секретом Рынок уже оценил эту новость и по сути дела это никак не ***отразилось*** на рынке А дальше рынок акций РАО ',
 'Однако по мнению Лабовского предстояще